In [1]:
# Example: Fetching connectivity SMILES from PubChemPy (no deprecation warning)
import pubchempy as pcp

compounds = pcp.get_compounds('aspirin', 'name')
for c in compounds:
    print(f"CID: {c.cid}, Connectivity SMILES: {c.connectivity_smiles}")

CID: 2244, Connectivity SMILES: CC(=O)OC1=CC=CC=C1C(=O)O


## Check for Existing Notebooks

Let's search the current directory for Jupyter notebook files.

In [2]:
import os

# List all .ipynb files in the current directory
notebooks = [f for f in os.listdir('.') if f.endswith('.ipynb')]
print("Notebooks in this directory:", notebooks)

Notebooks in this directory: ['chapter3-cheminformatics-essentials.ipynb', 'sample_generated_notebook.ipynb', 'chapter1-genomics-to-molecules.ipynb', 'chapter4-compound-screening.ipynb', 'chapter5-ml-for-drug-discovery.ipynb', 'chapter2-target-identification.ipynb']


## Generate a New Notebook File

You can programmatically create a new Jupyter notebook using the `nbformat` library.

In [3]:
import nbformat as nbf

nb = nbf.v4.new_notebook()
nb['cells'] = [nbf.v4.new_code_cell("print('Hello, new notebook!')")]
with open('sample_generated_notebook.ipynb', 'w') as f:
    nbf.write(nb, f)
print("Created sample_generated_notebook.ipynb")

Created sample_generated_notebook.ipynb


## List All Notebooks in the Directory

Let's display all `.ipynb` files in the current directory again.

In [4]:
# List all .ipynb files again
notebooks = [f for f in os.listdir('.') if f.endswith('.ipynb')]
print("Notebooks in this directory:", notebooks)

Notebooks in this directory: ['chapter3-cheminformatics-essentials.ipynb', 'sample_generated_notebook.ipynb', 'chapter1-genomics-to-molecules.ipynb', 'chapter4-compound-screening.ipynb', 'chapter5-ml-for-drug-discovery.ipynb', 'chapter2-target-identification.ipynb']


## Open a Notebook in Visual Studio Code

You can open a notebook from the terminal using:

```bash
code chapter2-target-identification.ipynb
```

## Run Code Cells Programmatically

You can execute notebook cells programmatically using `nbconvert` or `papermill`. Example with papermill:

```python
import papermill as pm
pm.execute_notebook('chapter2-target-identification.ipynb', 'output.ipynb')
```


## Practical Target Identification Examples

Below are three hands-on code examples for target identification using NLP and public databases. Each example includes an explanation to help you understand the workflow and its relevance to drug discovery.

1. **NER on Custom Biomedical Text:** Use a pretrained NLP model to extract gene/protein names (potential drug targets) from biomedical sentences.
2. **Automated PubMed Search and NER:** Search PubMed for disease-related abstracts, then use NER to extract targets from real literature.
3. **UniProt Query for Target Information:** Query UniProt for detailed information about a gene/protein target identified in previous steps.

In [5]:
# 1. NER on Custom Biomedical Text
from transformers import pipeline

# Use a public general NER model
nlp = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
text = "APP and BACE1 are key targets in Alzheimer's disease."
results = nlp(text)
print("Named Entities:", results)

Skipping import of cpp extensions due to incompatible torch version 2.9.1 for torchao version 0.13.0
W1127 14:55:11.293000 72956 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Named Entities: [{'entity_group': 'ORG', 'score': np.float32(0.56968904), 'word': 'AP', 'start': 0, 'end': 2}, {'entity_group': 'MISC', 'score': np.float32(0.67893267), 'word': 'BACE', 'start': 8, 'end': 12}, {'entity_group': 'MISC', 'score': np.float32(0.90901405), 'word': 'Alzheimer', 'start': 33, 'end': 42}]


In [6]:
# 2. Automated PubMed Search and NER
from Bio import Entrez
from transformers import pipeline

Entrez.email = "justinzhang.xl@gmail.com"  # Replace with your email

# Search PubMed for abstracts related to Alzheimer's targets
handle = Entrez.esearch(db="pubmed", term="Alzheimer target", retmax=1)
record = Entrez.read(handle)
pubmed_ids = record["IdList"]

# Fetch the abstract text
if pubmed_ids:
    fetch_handle = Entrez.efetch(db="pubmed", id=pubmed_ids[0], rettype="abstract", retmode="text")
    abstract = fetch_handle.read()
    print("Abstract:\n", abstract)
    # Run NER on the abstract using a public general NER model
    nlp = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
    entities = nlp(abstract)
    print("Named Entities:", entities)
else:
    print("No PubMed results found.")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Abstract:
 1. Neurol Ther. 2025 Nov 27. doi: 10.1007/s40120-025-00869-w. Online ahead of 
print.

Care Partners' Perceptions of Amyloid-Targeting Therapy and Treat‑to‑Clearance 
for Alzheimer's Disease in Japan: A Qualitative Study.

Katayama S(1), Tsujimoto M(2), Suzuki K(2), Ueda K(3), Shimoura K(4), Suo S(4), 
Hatakeyama N(5).

Author information:
(1)Katayama Medical Clinic, Matsuda Parking Building 1F, 1-26, Kotobuki-cho, 
Kurashiki-shi, Okayama, 710-0813, Japan.
(2)Innovation Center for Translational Research, National Center for Geriatrics 
and Gerontology, 7-430 Morioka-cho, Obu-shi, Aichi, 474-8511, Japan.
(3)Eli Lilly Japan K.K., LILLY PLAZA ONE BLDG, 5-1-28, Isogami-dori, Chuo-ku, 
Kobe-shi, Hyogo, 651-0086, Japan. ueda_kaname@lilly.com.
(4)Mebix, Inc., 1-11-44 Akasaka, Minato-ku, Tokyo, 107-0052, Japan.
(5)Eli Lilly Japan K.K., LILLY PLAZA ONE BLDG, 5-1-28, Isogami-dori, Chuo-ku, 
Kobe-shi, Hyogo, 651-0086, Japan.

INTRODUCTION: Donanemab has been developed as an amyloid-tar

Device set to use cpu


Named Entities: [{'entity_group': 'ORG', 'score': np.float32(0.99550205), 'word': 'N', 'start': 3, 'end': 4}, {'entity_group': 'ORG', 'score': np.float32(0.9862751), 'word': '##eurol Ther', 'start': 4, 'end': 14}, {'entity_group': 'ORG', 'score': np.float32(0.99838954), 'word': 'Care Partners', 'start': 87, 'end': 100}, {'entity_group': 'MISC', 'score': np.float32(0.6516541), 'word': '##id', 'start': 122, 'end': 124}, {'entity_group': 'MISC', 'score': np.float32(0.85668623), 'word': "Alzheimer ' s Disease", 'start': 171, 'end': 190}, {'entity_group': 'LOC', 'score': np.float32(0.9992847), 'word': 'Japan', 'start': 194, 'end': 199}, {'entity_group': 'PER', 'score': np.float32(0.83307457), 'word': 'Kat', 'start': 223, 'end': 226}, {'entity_group': 'ORG', 'score': np.float32(0.7528863), 'word': '##aya', 'start': 226, 'end': 229}, {'entity_group': 'PER', 'score': np.float32(0.6599837), 'word': 'S', 'start': 232, 'end': 233}, {'entity_group': 'PER', 'score': np.float32(0.9979234), 'word': '

In [7]:
# 3. UniProt Query for Target Information
import requests

def get_uniprot_by_accession(accession):
    url = f"https://rest.uniprot.org/uniprotkb/{accession}.tsv?fields=accession,protein_name,gene_names,organism_name,length"
    response = requests.get(url)
    print(f"Results for {accession} (UniProt):\n{response.text}")

# Example: Query UniProt for several Alzheimer's-related targets by accession
get_uniprot_by_accession("P05067")  # APP
get_uniprot_by_accession("P56817")  # BACE1
get_uniprot_by_accession("P10636")  # MAPT
get_uniprot_by_accession("P49768")  # PSEN1
get_uniprot_by_accession("P49810")  # PSEN2

Results for P05067 (UniProt):
Entry	Protein names	Gene Names	Organism	Length
P05067	Amyloid-beta precursor protein (APP) (ABPP) (APPI) (Alzheimer disease amyloid A4 protein homolog) (Alzheimer disease amyloid protein) (Amyloid precursor protein) (Amyloid-beta (A4) precursor protein) (Amyloid-beta A4 protein) (Cerebral vascular amyloid peptide) (CVAP) (PreA4) (Protease nexin-II) (PN-II) [Cleaved into: N-APP; Soluble APP-alpha (S-APP-alpha); Soluble APP-beta (S-APP-beta); C99 (Beta-secretase C-terminal fragment) (Beta-CTF); Amyloid-beta protein 42 (Abeta42) (Beta-APP42); Amyloid-beta protein 40 (Abeta40) (Beta-APP40); C83 (Alpha-secretase C-terminal fragment) (Alpha-CTF); P3(42); P3(40); C80; Gamma-secretase C-terminal fragment 59 (Amyloid intracellular domain 59) (AICD-59) (AID(59)) (Gamma-CTF(59)); Gamma-secretase C-terminal fragment 57 (Amyloid intracellular domain 57) (AICD-57) (AID(57)) (Gamma-CTF(57)); Gamma-secretase C-terminal fragment 50 (Amyloid intracellular domain 50) (AICD-5

### 1. NER on Custom Biomedical Text
This cell uses a pretrained BioBERT model to perform Named Entity Recognition (NER) on a biomedical sentence. NER helps identify gene and protein names mentioned in scientific text, which are potential drug targets. This is a common first step in literature mining for target identification.

### 2. Automated PubMed Search and NER
This cell demonstrates how to search PubMed for abstracts related to a disease (e.g., Alzheimer's), retrieve the abstract text, and then use NER to extract gene/protein names. This workflow automates the process of finding and prioritizing new drug targets from the latest scientific literature.

### 3. UniProt Query for Target Information
This cell shows how to query the UniProt database for detailed information about a gene or protein (such as APP). After identifying a potential target using NER and literature mining, you can use UniProt to learn about its biological function, sequence, and relevance to disease, helping you decide if it is a good candidate for drug development.